In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import nibabel as nib0
from scipy import ndimage

In [ ]:
input_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/"
test_images_dir = input_path + 'stage_2_test/'
train_images_dir = input_path + 'stage_2_train/'

In [ ]:
def read_nifti_file(input_path):
    """Read and load volume"""
    # Read file
    scan = nib.load(input_path)
    # Get raw data
    scan = scan.get_fdata()
    return scan

In [ ]:
def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

In [ ]:
def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

In [ ]:
def process_scan(input_path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(input_path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [ ]:
normal_scan_paths = [
    os.path.join(os.getcwd(), "../input/rsna-intracranial-hemorrhage-detection", x)
    for x in os.listdir("../input/rsna-intracranial-hemorrhage-detection")